In [4]:
import os
%cd ..
%cd ..
os.getcwd()

/Users/macbook/Documents/Code/uni-guider/src
/Users/macbook/Documents/Code/uni-guider


'/Users/macbook/Documents/Code/uni-guider'

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_huggingface import HuggingFaceEmbeddings
from src.chatbot.utils import *
import numpy as np
from pyvi.ViTokenizer import tokenize
from uuid import uuid4
# from rag.prompt import template
from src.chatbot.rag.config import LLM_CONFIG, EMBEDDING_CONFIG
from pydantic import BaseModel
from sentence_transformers import CrossEncoder
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama

In [6]:
data = load_doc_from_json(filename="data/document_langchain.json")

In [7]:
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=EMBEDDING_CONFIG["model_name"],
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

llm = ChatOllama(model = LLM_CONFIG["model_name"])

In [8]:
index = faiss.IndexFlatL2(len(hf.embed_query("Xin chào.")))
vectorstore = FAISS(
    embedding_function=hf,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

uuids = [str(uuid4()) for _ in range(len(data))]
vectorstore.add_documents(documents=data, ids=uuids)

retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [9]:
query = "Bảo hiểm y tế của Bách Khoa"
MODEL_ID = 'itdainb/PhoRanker'
MAX_LENGTH = 512
model = CrossEncoder(MODEL_ID, max_length=MAX_LENGTH)
model.model.half()

def retrieve_documents(query: str) -> List[Document]:
    docs = retriever.invoke(query)

    segmented_question = tokenize(query)
    segmented_documents = [tokenize(doc.page_content) for doc in docs]
    tokenized_pairs = [[segmented_question, sent] for sent in segmented_documents]

    scores = model.predict(tokenized_pairs)
    top_ids = np.argsort(scores)[::-1][:5]
    top_documents = [docs[i] for i in top_ids]
    return top_documents


In [10]:
template = '''Bạn là một cô gái hướng dẫn dễ thương tên là Mei Mei, đáng yêu đang trả lời thắc mắc của các bạn sinh viên Bách Khoa về trường của họ.
Bạn hãy trả lời lại thông tin sau đây theo giọng điệu đáng yêu nhí nhảnh của mình nhé.
Bạn chỉ được trả lời dựa theo ngữ cảnh đã được cung cấp và tuyệt đối không được trả lời vượt qua phạm vi ngữ cảnh.

'''

In [ ]:
while True:
    question = input("Bạn muốn hỏi gì: ")
    if question.lower() in ['/exit']:
        print("Tạm biệt nhé tình iu của Mei, hẹn gặp bạn lần sau!")
        break
    
    filtered_docs = retrieve_documents(question)
    doc_txts = "\n\n".join(doc.page_content for doc in filtered_docs)
    # prompt = rag_prompt.format(context = doc_txts)
    print(f"Length of doc is: {len(doc_txts.split())}")
    query = [SystemMessage(f"{template}\nNgữ cảnh: \n{doc_txts}"), HumanMessage(question)]
    response = llm.invoke(query)
    print(response.content)
    

In [2]:
from fastapi import FastAPI
from pydantic import BaseModel

app = FastAPI()

class QueryRequest(BaseModel):
    query: str

@app.post("/generate")
async def generate_response(request: QueryRequest):
    query = request.query
    llm = ChatOllama(model = "llama3.2:3b-instruct-fp16")
    response = llm.invoke(query)
    text_response = response.content
    return {"response": text_response}

In [5]:
x = 'ba la cay nen vang. me la cay nen! xanh.'
y = x.split()
y

['ba', 'la', 'cay', 'nen', 'vang.', 'me', 'la', 'cay', 'nen!', 'xanh.']